In [117]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

In [122]:
widths = [1]*32
df = pd.read_fwf('digitdata/optdigits-orig_train.txt', widths=widths, header=None, skiprows=lambda x : (x+1) % 33 == 0)
# df
df_Y = pd.read_fwf('digitdata/optdigits-orig_train.txt', widths=[2], header=None, skiprows=lambda x : (x+1) % 33 != 0)
overlap_x = 2
overlap_y = 2 # the window size  x for horizontal, y for vertical

In [123]:
df_temp = df.values.reshape(2436,32,32)
df_temp1 = np.empty((2436,33-overlap_x,33-overlap_y))
for index in range(2436):
    for i in range(33-overlap_y):
        for j in range(33-overlap_x):
            df_temp1[index,i,j] = df_temp[index,i,j] * 8 + df_temp[index,i,j+1]* 4
            + df_temp[index,i+1,j] * 2 + df_temp[index,i+1,j+1]
            # variant with window size
# df_temp1.shape

In [ ]:
df_trainX = pd.DataFrame(df_temp1.reshape(2436, (33-overlap_x)*(33-overlap_y)))
df_trainY = df_Y
df_trainY.columns = ['label']
# df_Y


In [ ]:
df = pd.read_fwf('digitdata/optdigits-orig_test.txt', widths=widths, header=None, skiprows=lambda x : (x+1) % 33 == 0)
df_Y = pd.read_fwf('digitdata/optdigits-orig_test.txt', widths=[2], header=None, skiprows=lambda x : (x+1) % 33 != 0)

df_temp = df.values.reshape(444,32,32)
df_temp1 = np.empty((444,33-overlap_x,33-overlap_y))
for index in range(444):
    for i in range(33-overlap_y):
        for j in range(33-overlap_x):
            df_temp1[index,i,j] = df_temp[index,i,j] * 8 + df_temp[index,i,j+1]* 4
            + df_temp[index,i+1,j] * 2 + df_temp[index,i+1,j+1]
df_temp1.shape

In [ ]:
df_temp = df_temp1.reshape(444, (33-overlap_x)*(33-overlap_y))
# df_temp.shape
temp = np.empty((16,444,(33-overlap_x)*(33-overlap_y)))
for val in range(16):
    feature_makeval = np.copy(df_temp)
    feature_makeval[df_temp != val] = 0
    feature_makeval[df_temp == val] = 1
    temp[val] = np.copy(feature_makeval)

df_test = np.array(temp).transpose((1,2,0)).reshape(444, 16*(33-overlap_x)*(33-overlap_y))
df_testX = pd.DataFrame(df_test)
df_testY = df_Y
df_testY.columns = ['label']

In [ ]:
class_count = np.array((df_trainY['label'].value_counts().sort_index()))
class_count

In [ ]:
class_log_prior = np.log(class_count) - np.log(len(df_trainY))
class_log_prior

In [ ]:
# df_train_make1 = np.copy(df_trainX)
# df_train_make1[df_trainX>0] = 1
# df_train_for_count = pd.DataFrame(df_train_make1)
# feature_count = np.array([[sum(df_train_for_count[df_trainY['label'] == i].iloc[:,j]) for i in range(10)] for j in range((33-overlap_x)*(33-overlap_y))]).T
# feature_count
feature_MF = np.empty((16,10,(33-overlap_x)*(33-overlap_y)))
# for val in range(16):
val = 1
    feature_makeval = np.copy(df_trainX)
    feature_makeval[df_trainX != val] = 0
    feature_makeval[df_trainX == val] = 1
    feature_count = np.array([[sum(feature_makeval[df_trainY['label'] == i][:,j]) for i in range(10)] for j in range((33-overlap_x)*(33-overlap_y))]).T
    feature_MF[val] = np.copy(feature_count)


In [ ]:
feature_count = feature_MF.transpose(1,2,0).reshape(10,(33-overlap_x)*(33-overlap_y)*16)
# len(feature_MF)
print('finished')

In [ ]:
def calc_feature_log_prob(feature_count, k=1):
    smoothed_fc = feature_count + k
    smoothed_cc = smoothed_fc.sum(axis=1)
    print(smoothed_cc.reshape(-1,1))
    return (np.log(smoothed_fc) - np.log(smoothed_cc.reshape(-1, 1)))
#########got trouble in making up posteriori###############
###########################################################

In [ ]:
feature_log_prob = [calc_feature_log_prob(feature_count, 0.1), \
                    calc_feature_log_prob(feature_count, 1), \
                    calc_feature_log_prob(feature_count, 5), \
                    calc_feature_log_prob(feature_count, 10)]
# feature_log_prob = calc_feature_log_prob(feature_count, 0.1)

In [ ]:
joint_log_likelihood = [df_testX @ feature_log_prob[0].T + class_log_prior, \
                        df_testX @ feature_log_prob[1].T + class_log_prior, \
                        df_testX @ feature_log_prob[2].T + class_log_prior, \
                        df_testX @ feature_log_prob[3].T + class_log_prior]
# joint_log_likelihood = df_testX @ feature_log_prob.T + class_log_prior

In [ ]:
pred = [np.argmax(joint_log_likelihood[0], axis=1), \
        np.argmax(joint_log_likelihood[1], axis=1), \
        np.argmax(joint_log_likelihood[2], axis=1), \
        np.argmax(joint_log_likelihood[3], axis=1)]
# pred = np.argmax(joint_log_likelihood, axis=1)

In [120]:
# df_testY.shape

In [121]:
smooth_k = [0.1, 1, 5, 10]
for i in range(4):
    y_pred = pd.Series(pred[i])
    df_confusion = pd.crosstab(df_testY['label'], y_pred)

    conf_percent = df_confusion.values / df_confusion.sum(axis=1).values.reshape(-1,1)
    df_conf_percent = round(pd.DataFrame(conf_percent).rename_axis('Actual').rename_axis('Prediction', axis=1), 4)*100
    print('Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = {})'.format(smooth_k[i]))
    display(df_conf_percent)
    display(pd.DataFrame(np.diag(df_conf_percent), columns=['Accuracy']))
    print('Average Accuracy: {}'.format(np.mean(np.diag(df_conf_percent))))
    print('\n')
# df_conf_percent

Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = 0.1)


Prediction,0,1,2,3,4,5,6,7,8,9
Actual,,,,,,,,,,
0,100.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
1,0.0,97.78,0.00,0.00,0.00,0.0,0.00,2.22,0.00,0.00
2,0.0,0.00,85.37,0.00,0.00,0.0,0.00,2.44,9.76,2.44
3,0.0,0.00,0.00,90.91,0.00,0.0,0.00,3.03,0.00,6.06
4,0.0,0.00,0.00,0.00,88.14,0.0,0.00,8.47,3.39,0.00
5,0.0,0.00,0.00,0.00,0.00,93.1,0.00,0.00,0.00,6.90
6,0.0,0.00,0.00,0.00,2.33,0.0,97.67,0.00,0.00,0.00
7,0.0,0.00,0.00,0.00,0.00,0.0,0.00,97.87,2.13,0.00
8,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,100.00,0.00


,Accuracy
0,100.00
1,97.78
2,85.37
3,90.91
4,88.14
5,93.10
6,97.67
7,97.87
8,100.00
9,92.86


Average Accuracy: 94.37


Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = 1)


Prediction,0,1,2,3,4,5,6,7,8,9
Actual,,,,,,,,,,
0,100.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
1,0.0,95.56,0.00,0.00,0.00,0.0,0.00,2.22,0.00,2.22
2,0.0,0.00,85.37,0.00,0.00,0.0,0.00,2.44,9.76,2.44
3,0.0,0.00,0.00,90.91,0.00,0.0,0.00,3.03,0.00,6.06
4,0.0,0.00,0.00,0.00,86.44,0.0,0.00,10.17,3.39,0.00
5,0.0,0.00,0.00,0.00,0.00,93.1,0.00,0.00,0.00,6.90
6,0.0,0.00,0.00,0.00,2.33,0.0,97.67,0.00,0.00,0.00
7,0.0,0.00,0.00,0.00,0.00,0.0,0.00,97.87,2.13,0.00
8,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,100.00,0.00


...

,Accuracy
0,100.00
1,95.56
2,85.37
3,96.97
4,84.75
5,93.10
6,97.67
7,97.87
8,97.50
9,85.71


Average Accuracy: 93.45


Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = 10)


Prediction,0,1,2,3,4,5,6,7,8,9
Actual,,,,,,,,,,
0,100.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
1,0.0,95.56,0.00,0.00,0.00,0.0,0.00,2.22,0.00,2.22
2,0.0,0.00,85.37,0.00,0.00,0.0,0.00,2.44,9.76,2.44
3,0.0,0.00,0.00,96.97,0.00,0.0,0.00,0.00,0.00,3.03
4,0.0,0.00,0.00,0.00,84.75,0.0,0.00,11.86,3.39,0.00
5,0.0,0.00,0.00,0.00,0.00,93.1,0.00,0.00,0.00,6.90
6,0.0,0.00,0.00,0.00,2.33,0.0,97.67,0.00,0.00,0.00
7,0.0,0.00,0.00,0.00,0.00,0.0,0.00,97.87,2.13,0.00
8,0.0,2.50,2.50,0.00,0.00,0.0,0.00,2.50,90.00,2.50


,Accuracy
0,100.00
1,95.56
2,85.37
3,96.97
4,84.75
5,93.10
6,97.67
7,97.87
8,90.00
9,85.71


Average Accuracy: 92.7


